In [44]:
%pip install redis

import redis

r = redis.Redis(host='localhost', port=6379, db=0)


Note: you may need to restart the kernel to use updated packages.


In [50]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
bank_id

'bank_40'

In [46]:
r.keys()

[b'faiss_index:v1::bank_40',
 b'faiss_index:v1::bank_39',
 b'faiss_index:v1::bank_38',
 b'memory_banks:v1::bank_38',
 b'memory_banks:v1::bank_39',
 b'memory_banks:v1::bank_40']

In [47]:
r.get(f"memory_banks:v1::{bank_id}")

b'{"identifier":"bank_40","provider_resource_id":"bank_40","provider_id":"faiss","type":"memory_bank","memory_bank_type":"vector","embedding_model":"all-MiniLM-L6-v2","chunk_size_in_tokens":512,"overlap_size_in_tokens":null}'

In [6]:
r.info()

{'redis_version': '7.4.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': '7fd3520e9c14a41b',
 'redis_mode': 'standalone',
 'os': 'Linux 6.8.0-49-generic x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '11.4.0',
 'process_id': 184895,
 'process_supervised': 'systemd',
 'run_id': 'a2cdc0a08c4d4eb4607a4792fb4e962e225ee8a6',
 'tcp_port': 6379,
 'server_time_usec': 1733637871975374,
 'uptime_in_seconds': 4488,
 'uptime_in_days': 0,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 5584623,
 'executable': '/usr/bin/redis-server',
 'config_file': '/etc/redis/redis.conf',
 'io_threads_active': 0,
 'listener0': {'name': 'tcp', 'bind': '-::1', 'port': 6379},
 'connected_clients': 3,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 8,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'pubsub_clients': 0,
 'watch

In [48]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

6941

In [51]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content='Hiccup fights to get free, finally springing to his feet and\n          shaking off the viscous saliva.\n           HTTYD 2 (D. DeBlois) - FINAL DRAFT - 05/05/14 10.\n\n                         \n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           You KNOW that doesn\'t wash out.\n\n                         \n          Toothless LAUGHS, pleased with himself.\n\n                         \n          Shifting his attention back to the new land stretched out\n          before them, Hiccup reaches beneath his chest plate and\n          produces a leather-bound book.\n\n                         \n          He unfurls it to reveal an EXPLORERS MAP, modified with added\n          strips in every direction to accommodate their ever-expanding\n          world. Drawings indicate new lands, new tribes, and many new\n          dragons.\n          Hiccup peels a fresh strip of paper from a hidden fold in his\n    

Chunk(content='Hiccup fights to get free, finally springing to his feet and\n          shaking off the viscous saliva.\n           HTTYD 2 (D. DeBlois) - FINAL DRAFT - 05/05/14 10.\n\n                         \n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           You KNOW that doesn\'t wash out.\n\n                         \n          Toothless LAUGHS, pleased with himself.\n\n                         \n          Shifting his attention back to the new land stretched out\n          before them, Hiccup reaches beneath his chest plate and\n          produces a leather-bound book.\n\n                         \n          He unfurls it to reveal an EXPLORERS MAP, modified with added\n          strips in every direction to accommodate their ever-expanding\n          world. Drawings indicate new lands, new tribes, and many new\n          dragons.\n          Hiccup peels a fresh strip of paper from a hidden fold in his\n          flight suit and holds it 

In [52]:
import pandas as pd

datasets = {
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,50.000000
mean,217480.920000
std,58733.594823
min,110830.000000
25%,186267.000000
50%,215859.500000
75%,240608.500000
max,417011.000000


In [53]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

1490
1490


['Who planned the robbery that was being investigated when Domino Harvey was arrested?',
 'Which career was Domino in before she became a bounty hunter?',
 "What did Lateesha plan to use to pay for her granddaughter's medical procedure?",
 'How did Claremont know there would be $10 Million in the armored car?',
 'Who were the FBI initially led to believe would commit the robbery?']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [54]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

100%|██████████| 1490/1490 [04:40<00:00,  5.31it/s]


[[0, 0, 0, 0, 45, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 17, 2, 0, 28, 16, 17],
 [35, 0, 0, 10, 37, 0, 0, 0, 0, 21],
 [45, 45, 23, 48, 0, 28, 4, 38, 45, 0],
 [0, 0, 16, 0, 15, 25, 0, 10, 38, 0],
 [0, 0, 10, 0, 0, 0, 10, 28, 35, 45],
 [34, 0, 18, 34, 34, 18, 34, 0, 18, 34],
 [10, 10, 0, 28, 28, 45, 24, 0, 10, 44],
 [0, 0, 0, 0, 0, 0, 35, 0, 35, 35],
 [0, 0, 0, 0, 0, 0, 0, 0, 16, 0],
 [2, 48, 9, 35, 18, 42, 0, 2, 38, 26],
 [0, 0, 0, 29, 0, 0, 0, 35, 10, 10],
 [0, 0, 0, 0, 0, 0, 0, 10, 10, 10],
 [0, 0, 0, 45, 0, 0, 36, 48, 21, 0],
 [0, 0, 29, 12, 29, 48, 10, 35, 10, 48],
 [45, 24, 44, 10, 4, 10, 17, 10, 34, 10],
 [0, 0, 0, 11, 0, 38, 0, 19, 39, 0],
 [0, 0, 22, 0, 18, 0, 14, 34, 34, 18],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [31, 49, 49, 0, 49, 49, 32, 49, 49, 26],
 [0, 21, 15, 21, 35, 28, 28, 31, 0, 0],
 [0, 45, 0, 0, 15, 44, 0, 38, 45, 4],
 [16, 16, 49, 22, 22, 0, 36, 0, 44, 16],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 45, 0, 0, 0, 0,

In [55]:
import numpy as np

targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1490,)

(1490, 1)

(1490, 10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
top_k = 3

res = np.array(results)
res = res[:, :top_k]
targets = targets[:, :top_k]
display(targets.shape)
display(res.shape)

(1490, 3)

(1490, 3)

In [65]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.019686800894854584)

In [66]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.05906040268456376)

In [67]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.03646532438478747)